## Importing Libraries and load Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os

In [ ]:
# os.listdir()
data = pd.read_excel('TrainDataset2023.xls')
cols = data.columns.values

# Checking for improper column names
bad_names = [name for name in cols if re.findall(r"\W",name)]
repaired_names = [re.sub(r'\s|\(.+\)','',name) for name in cols]


# Rename columns
data.columns = repaired_names
data.columns


# Data processing

In [ ]:
# check duplicated rows
# No duplicates were found in data
duplicates = data.duplicated()
# duplicates[duplicates==True]
duplicates.any()

In [ ]:
# Checking for missing values
# Missing values are coded as 999

for col in data.columns:
    data[col].replace(999,np.nan, inplace=True)

In [ ]:
def check_missing(df):
    missing = df.isna().sum()
    missing = missing[missing>0]
    missing.sort_values(ascending=False,inplace=True)
    # print(missing)
    return(missing)

check_missing(data)

In [ ]:
# Since the target is outcome of treatment, imputing this variable may not be safe, therefore dropped from the dataset
# missing values in features can be imputed
data = data.loc[~data['pCR'].isna(),:]

# again check for missing values
check_missing(data)

# Checking unique values for each column
- This basically identifies categorical features in data, since no text labels were provided
- experiment with different thresholds to confirm

In [ ]:
count_uni = dict()
for i,col in enumerate(data.columns.values):
    count_uni[col] = len(np.unique(data.loc[:,col]))

# print(count_uni)

list_cat = []
for key,val in count_uni.items():
    if count_uni[key]<10:
        list_cat.append(key)
        # print(f"{key}:{val}")

print(f"There are : {len(list_cat)} categotical fetures: {list_cat}")
    

In [ ]:
# drop ID column
# data.drop('ID',axis=1,inplace=True)
# data.columns

## Train test split
- Dataset is unbalanced, around 78 percent are negative samples for `pCR`. Therefore it is necessary that train-test split produce represenative sets 
- Train test split is done before even exploring the dataset! to ensure that the test set is unseen to avoid so called data snooping bias. Notewortthy, seeing full dataset before training may influence choice of particular algorithm, thus prone to overfitting to data. 


In [ ]:
## Split the dataset into training and test sets, before pre-processing to avoid data leakage
from sklearn.model_selection import train_test_split


# For classification task
data_train, data_test = train_test_split(data,test_size=.20,shuffle=True,stratify=data['pCR'],random_state=0)

print(data_train.index)
# Lets rename our training data as df to avoid any confusion in subsequent stages
# From now nowards the training set will be df, and we have reserved `data_test` for evaluation
# df will be used for exploratory analysis only
df = data_train.copy()
df.drop('ID',axis=1,inplace=True)


## EXPORT TEST DATA IN PRESCRIBED FORMAT

In [ ]:
test_set_export = data_test.loc[:,~data_test.columns.isin(['pCR','RelapseFreeSurvival'])]
test_set_export.to_excel('test_set_PCR.xlsx',index=False)
test_set_export

In [ ]:
# check distribution (proportions of positive vs negative samples) of data by pCR
p_df = df['pCR'].value_counts()/sum(df['pCR'].value_counts())
p_test = data_test['pCR'].value_counts()/sum(data_test['pCR'].value_counts())

print(p_df)
print(df.shape)
print(p_test)
print(data_test.shape)

## Imputing missing values
- KNN imputer is preferred since it imputes values based on closely similar datapoints
- One neigbor is used for imputation, this avoids meaningless imputation for categorical features
- To avoid data leakage, a test set is reserved before any data preprocessing
- This will be used to evaluate the models after training is completed
- imputation done here is just for exploratoty analysis

In [ ]:
# replace missing values by imputation, 

from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=1)

try:
    tmp = imputer.fit_transform(df)
except KeyError:
    print('Variable not found')

imputer.feature_names_in_
imputer.n_features_in_

df_imp = pd.DataFrame(tmp,columns=imputer.get_feature_names_out())
# X_train.head()

print(df_imp.shape)
print(df.shape)

# check missing
check_missing(df_imp)

# df_imp = df.copy()

In [ ]:
# Describe continuous features
df_imp.loc[:,~df_imp.columns.isin(list_cat+['RelapseFreeSurvival'])].describe()

In [ ]:
# Counts for target `pCR`and categorical features

for v in list_cat:
    print(df_imp[v].value_counts())

#data['pCR'].value_counts()

In [ ]:
# change categorical features to int for better hist plots
# data_cat = df_imp[list_cat]
# data_types =  data_cat.dtypes

# for feature in list_cat:
#     # if data_cat[feature].dtype!=object:
#     data_cat.loc[:,feature] = data_cat[feature].astype(int)


In [ ]:
# change categorical features to int for better hist plots
data_cat = df_imp[list_cat]
data_types =  data_cat.dtypes

for feature in list_cat:
    # if data_cat[feature].dtype!=object:
    data_cat.loc[:,feature] = data_cat[feature].astype('int64')


## Categorical features vs `pCR`
- There are observed differences in counts between negative and positive samples for pGR, HistrologyType, TumorStage, etc..
- To be confirmed later by statistical tests, to ensure the observed differences are not by chance

In [ ]:

fig, axs = plt.subplots(5,2,figsize=(10,20))
for ax, series in zip(axs.ravel(),data_cat):
    sns.countplot(x=series,ax=ax, data=data_cat,hue='pCR')
# axs[0,0].set_title('ER')


## Visualizing continuous data
- so many features have outliers, these need be removed
- some are so huge, potentially indicating data entry errors

In [ ]:
list_cont = [i for i in df_imp.columns.values if i not in list_cat]
data_cont = df_imp.loc[:,list_cont]


fig, axs = plt.subplots(10,11,figsize=(100,60))
for ax, series in zip(axs.ravel(),data_cont):
    sns.boxplot(x=series,ax=ax, data=data_cont)




In [ ]:

fig, axs = plt.subplots(8,8,figsize=(25,20))
axs = axs.ravel()
for i,col in enumerate(data_cont.iloc[:,0:64].columns.values):
    axs[i].hist(x=col,bins=50,data=data_cont.iloc[:,0:64])
    axs[i].set_title(col)
plt.tight_layout()

In [ ]:
len(axs)

In [ ]:
# data_cont.iloc[:,64:data_cont.shape[1]].hist(bins=50,figsize=(30,25))
fig, axs = plt.subplots(8,6,figsize=(25,20))
axs = axs.ravel()
for i,col in enumerate(data_cont.iloc[:,64:].columns.values):
    axs[i].hist(x=col,bins=50,data=data_cont.iloc[:,64:])
    axs[i].set_title(col)
plt.tight_layout()

In [ ]:
# Pairs plots
# add pcr to color pairs plots to check more clearly
list_cont_pcr = list_cont[:]
list_cont_pcr.pop(list_cont_pcr.index('RelapseFreeSurvival'))
list_cont_pcr.insert(0,'pCR')
df.loc[:,list_cont_pcr]

sns.pairplot(data=df.loc[:,list_cont_pcr[0:11]],hue='pCR')


In [ ]:
l2 = list_cont_pcr[11:21]
l2.insert(0,'pCR')
sns.pairplot(data=df.loc[:,l2],hue='pCR')

## Comparing feature ranges
 - Huge difference between feature ranges, normalization necessary

In [ ]:

fig,ax=plt.subplots(1,figsize=(40,8))
sns.boxplot(data=df.iloc[:,1:50],ax=ax)

## Removing outliers
- Identify them by using factor 1.5*IQR, 
Ref: Brownlee, J., 2020. Data preparation for machine learning: data cleaning, feature selection, and data transforms in Python. Machine Learning Mastery.
- lets replace them with median values!

In [ ]:
class OutlierRemover:
    """
    Takes `dataframe` as an input and list of continuous features in data frame 'cont_features_list'
    to get a list of features with outliers pass `.features_with_outliers` attribute to OutlierRemover object instance
    """
    def __init__(self,dataframe,cont_features_list):
        self.dataframe = dataframe
        self.cont_features_list = cont_features_list


    def transform(self,factor=3):
        """
        Factor is a multiplying factor to the interquartile range (IQR), cutoff = factor*IQR. Default is 3 for extreme outliers, 1.5 can be used.
        An outlier is identified as feature value that is larger than 3rd quatile + cutoff or less 1st quartile - cutoff value.
        """
        data_cont = self.dataframe.loc[:,self.dataframe.columns.isin(self.cont_features_list)]
        p25, p50, p75 = np.nanquantile(data_cont,.25,axis=0),np.nanquantile(data_cont,.5,axis=0),np.nanquantile(data_cont,.75,axis=0)

        cutoff = factor*(p75-p25) 
        lower, upper = p25 - cutoff, p75 + cutoff

        dict_upper = dict()
        dict_lower = dict()
        dict_med = dict()
        for low, up, med,col in zip(lower,upper,p50,data_cont.columns.values):
            dict_lower[col] = low
            dict_upper[col] = up
            dict_med[col] = med


        self.features_with_outliers = []
        for col in data_cont.columns.values:
            if any(data_cont[col]<dict_lower[col]) or any(data_cont[col]>dict_upper[col]):
                self.features_with_outliers.append(col)

        # for col in data_cont.columns.values:
        for col in self.features_with_outliers:
            data_cont.loc[data_cont[col]>dict_upper[col],col] = dict_med[col]
            data_cont.loc[data_cont[col]<dict_lower[col],col] = dict_med[col]

        # Combine transformed features back to original data
        df1 = self.dataframe.loc[:,~self.dataframe.columns.isin(list_cont)]
        
        return pd.merge(df1,data_cont,left_index=True,right_index=True)

## Continuous features vs `pCR`
- not much to see through, statistical tests will complement

In [ ]:
ourm = OutlierRemover(data_cont,list_cont)
data_cont_ourm = ourm.transform()
# Visualize boxplots
fig, axs = plt.subplots(11,10,figsize=(40,30))
ax = axs.ravel()
df_cont_pcr = pd.merge(data_train.loc[:,['pCR']],data_cont_ourm,left_index=True,right_index=True)
for i,j in enumerate(df_cont_pcr.columns.values[1:]):
    sns.boxplot(x = df_cont_pcr['pCR'],y = df_cont_pcr.iloc[:,1:].loc[:,j],ax=ax[i],showmeans=True)

## ANOVA Test
- This will later be incorporated in pipeline for feature selection
- Aova test for two groups, is equivalent to t-test

In [ ]:
## ANOVA test
from sklearn.feature_selection import f_classif, SelectKBest
X_anova = data_cont_ourm.iloc[:,1:]
y_anova=data_cat.loc[:,'pCR']

fstats, pvalues = f_classif(X_anova,y_anova)

fselector = SelectKBest(f_classif, k=10)
fselector.fit_transform(X_anova,y_anova)

print(fselector.get_feature_names_out())
print(pvalues[pvalues<0.05])


# print(f"selected features: {X_anova.columns.values[fselector.get_support()]}")
# print(pvalues)



## Chi-square test for categorical feature selection
 - Only 3 fetures returned a statistically significant test at 5% significance level

In [ ]:
from sklearn.feature_selection import chi2
X_chi = df_imp.loc[:,list_cat[1:]]
y_chi = df_imp.loc[:,['pCR']]
chi2_selector = SelectKBest(chi2,k=3)
chi2_selector.fit_transform(X=X_chi,y=y_chi)
pvalues = chi2_selector.pvalues_

print(pvalues[pvalues<0.05])

print(chi2_selector.get_feature_names_out())

## Logistic classifier

In [ ]:
# converts floats to intergers for categorical data
def to_int(df,list_features):
    for f in list_features:
        df[f]=df[f].astype('Int64')
    return df

data_train = to_int(data_train,list_cat)
data_train
data_test = to_int(data_test,list_cat)
data_test

In [ ]:
# Apply robust scaler to image data to scale them and remove outliers
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold, RepeatedStratifiedKFold,RepeatedKFold, cross_val_score,cross_val_predict
from sklearn.metrics import balanced_accuracy_score,confusion_matrix,classification_report,roc_curve,auc
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.impute import KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer #For custom transformation functions
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


target = 'pCR'
features_to_exlude_in_x = ['ID','pCR','RelapseFreeSurvival']

def split_x_y(dataframe,features_to_exlude_in_x,target):
    features_to_exlude_in_x.append(target)
    y = dataframe.loc[:,target]
    X = dataframe.loc[:,~dataframe.columns.isin(features_to_exlude_in_x)]
    return X, y

X,y = split_x_y(data_train ,features_to_exlude_in_x,target)

# Test data
X_test, y_test = split_x_y(data_test,features_to_exlude_in_x,target)
Xcols = X_test.columns.values

# categorical features to select from 
list_cat[1:]


imputer = KNNImputer(n_neighbors=1)

cont_pipeline = Pipeline(steps=[('imputer',imputer),
                                ('scaler',RobustScaler()),
                                ('kbest_cont',SelectKBest(f_classif,k=9))])

cat_pipleline = Pipeline(steps=[('imputer',imputer),
                                ('kbest_cat',SelectKBest(chi2,k=4))])

transformer = ColumnTransformer(transformers=[('cont',cont_pipeline,list_cont[1:]),
                                              ('cat',cat_pipleline,list_cat[1:])
                                            ])

regressor = LogisticRegression(max_iter=10_000,class_weight='balanced',random_state=0)
clf_lr = Pipeline(steps=[('processor',transformer),
                         ('regressor',regressor)])

param_grid = {
    'regressor__C':[0.01,0.05,1,2,5,10,20]
    # 'processor__cont__kbest_cont__k':list(range(1,10))
}

cv  = RepeatedStratifiedKFold(n_splits=5,n_repeats=5,random_state=0)
grid_search = GridSearchCV(estimator=clf_lr,param_grid=param_grid,cv=cv,n_jobs=-1,scoring='balanced_accuracy')

grid_search.fit(X,y)
clf_lr = grid_search.best_estimator_
ba_cv_clf_lr =cross_val_score(clf_lr,X,y,scoring='balanced_accuracy')

y_pred = clf_lr.predict(X_test)
ba_tst_clf_lr = balanced_accuracy_score(y_true=y_test,y_pred=y_pred)


print(f"BA CV: {np.mean(ba_cv_clf_lr):.3f}, cv_sd: {np.std(ba_cv_clf_lr):.3f}, BA Test: {ba_tst_clf_lr:.3f}")

report=classification_report(y_true=y_test,y_pred=y_pred)
print(report)
clf_lr

In [ ]:
## Storing model performance metrics
def model_performance(name,cross_validation_score,test_score):
    metrics = {'name':name,'ba_cv':np.mean(cross_validation_score),'sd':np.std(cross_validation_score),'ba_test':test_score}
    return metrics

perfopmance_clf_lr= model_performance('Logistic',ba_cv_clf_lr,ba_tst_clf_lr)

In [ ]:
grid_search.best_params_

In [ ]:
# # user function for plotting
# def plot_roc_curve(model):
#     y_proba = model.predict_proba(X_test)
#     fpr, tpr, _ = roc_curve(y_true=y_test,y_score=y_proba[:,1])
#     roc_auc = auc(fpr, tpr)

#     plt.figure()
#     lw = 2
#     plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
#     plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
#     plt.xlim([0.0, 1.0])
#     plt.ylim([0.0, 1.05])
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.title('Receiver Operating Characteristic Curve')
#     plt.legend(loc="lower right")
#     plt.show()
# plot_roc_curve(clf_lr)


## Support Vector Machine classifier

In [ ]:

from sklearn.utils import resample
from sklearn.svm import SVC
# Define the parameters to tune
parameters_svm = {'svm__C':[0.01,0.05,1,2,5,20],
                  'svm__kernel':['sigmoid']}
# Bootstrapping
ba_cv_scores_svm = []
ba_tst_scores = []
for i in range(150):  # Number of bootstrap samples to create
    X_sample, y_sample = resample(X, y, replace=True)
    
    # Define the model
    svm = SVC(class_weight='balanced')
    clf_svm = Pipeline(steps=[('processor',transformer),
                             ('svm',svm)])
    
    # Define the grid search
    cv  = RepeatedStratifiedKFold(n_splits=10,n_repeats=3)
    grid_search_svm = GridSearchCV(estimator=clf_svm,param_grid=parameters_svm,cv=cv,n_jobs=-1,scoring='balanced_accuracy')
    
    # Fit the grid search
    grid_search_svm.fit(X_sample, y_sample)
    svm = grid_search_svm.best_estimator_
    # Calculate the balanced accuracy with cross-validation
    ba_cv_svm =cross_val_score(svm,X_sample,y_sample,scoring='balanced_accuracy')
    ba_cv_scores_svm.append(np.mean(ba_cv_svm))

# Predict on the test set and calculate the balanced accuracy
y_pred_svm = svm.predict(X_test)
ba_tst_svm = balanced_accuracy_score(y_true=y_test,y_pred=y_pred_svm)


# Calculate the average balanced accuracy over all bootstrap samples
average_ba_cv_svm = np.mean(ba_cv_svm)
# average_ba_tst = np.mean(ba_tst_scores)

print(f"Average BA CV, svm: {average_ba_cv_svm:.3f}, Average BA Test, svm: {ba_tst_svm:.3f}")
svm

In [ ]:
from sklearn.svm import SVC

# Bootstrapping
imputer = KNNImputer(n_neighbors=1)

cont_pipeline = Pipeline(steps=[('imputer',imputer),
                                ('scaler',RobustScaler()),
                                ('kbest_cont',SelectKBest(f_classif))])

cat_pipleline = Pipeline(steps=[('imputer',imputer),
                                ('kbest_cat',SelectKBest(chi2,k=2))])

transformer = ColumnTransformer(transformers=[('cont',cont_pipeline,list_cont[1:]),
                                              ('cat',cat_pipleline,list_cat[1:])
                                            ])


# Define the model
svm = SVC(class_weight='balanced',probability=True, random_state=0)
clf_svm = Pipeline(steps=[('processor',transformer),
                            ('svm',svm)])

# Define the grid search
# Define the parameters to tune
parameters_svm = {'svm__C':[0.01,0.05,1,2,5,20],
                  'processor__cont__kbest_cont__k':list(range(1,11)),
                  'processor__cat__kbest_cat__k':list(range(1,5)),
                  'svm__kernel':['sigmoid']}
cv  = RepeatedStratifiedKFold(n_splits=10,n_repeats=3)
grid_search_svm = GridSearchCV(estimator=clf_svm,param_grid=parameters_svm,cv=cv,n_jobs=-1,scoring='balanced_accuracy')

# Fit the grid search
grid_search_svm.fit(X, y)
svm = grid_search_svm.best_estimator_
# Calculate the balanced accuracy with cross-validation
ba_cv_svm =cross_val_score(svm,X,y,scoring='balanced_accuracy')


# Predict on the test set and calculate the balanced accuracy
y_pred = svm.predict(X_test)
ba_tst_svm = balanced_accuracy_score(y_true=y_test,y_pred=y_pred)


# Calculate the average balanced accuracy over all bootstrap samples
average_ba_cv_svm = np.mean(ba_cv_svm)
# average_ba_tst = np.mean(ba_tst_scores)

print(f"Average BA CV, svm: {np.mean(ba_cv_svm):.3f}, {np.std(ba_cv_svm):.3f} BA Test, svm: {ba_tst_svm:.3f}")

report=classification_report(y_true=y_test,y_pred=y_pred)
print(report)
svm

In [ ]:
perfopmance_reg_svm_smote = model_performance('SVM SMOTE',ba_cv_svm,ba_tst_svm)


## Random Forest Classifier
- Does not require feature selection, since by its design has inbuit feature selection methods
- Thus model is trained with all features included, redundant features will be dropped automatically


In [ ]:
# Apply robust scaler to image data to scale them and remove outliers
# With feature selection
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold, RepeatedStratifiedKFold,RepeatedKFold, cross_val_score,cross_val_predict
from sklearn.metrics import balanced_accuracy_score,accuracy_score,confusion_matrix
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.compose import TransformedTargetRegressor
from imblearn.ensemble import BalancedRandomForestClassifier

imputer = KNNImputer(n_neighbors=1)

clf_rf = BalancedRandomForestClassifier(200,random_state=0,bootstrap=True,class_weight='balanced_subsample')

model_rf = Pipeline(steps=[('imputer',imputer),('classifier',clf_rf)])


param_grid = param_grid = {#'classifier__max_features': [2,4,6,8,13],
                           'classifier__max_depth': [2,3,4,6]}

cv = RepeatedStratifiedKFold(n_repeats=4,n_splits=10,random_state=0)
grid_search = GridSearchCV(estimator=model_rf,param_grid=param_grid,cv=cv,scoring='balanced_accuracy',n_jobs=-1)
grid_search.fit(X=X,y=y)
bal_random_forest = grid_search.best_estimator_
best_parameters = grid_search.best_params_

ba_cv_rf = cross_val_score(bal_random_forest,X,y,scoring='balanced_accuracy')


# Predict and test
y_pred = bal_random_forest.predict(X_test)
ba_test_rf = balanced_accuracy_score(y_true=y_test,y_pred=y_pred)


print(f"BA CV: {np.mean(ba_cv_rf):.3f}, cv_sd: {np.std(ba_cv_rf):.3f}, BA Test: {ba_test_rf:.3f}")
report=classification_report(y_true=y_test,y_pred=y_pred)
print(report)
bal_random_forest

In [ ]:
perfopmance_bal_rf = model_performance('Random Forest',ba_cv_rf,ba_test_rf)


## XGBoost Classifier

In [ ]:
# simple xgboost
# Apply robust scaler to image data to scale them and remove outliers
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold, RepeatedStratifiedKFold,RepeatedKFold, cross_val_score,cross_val_predict
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from sklearn.preprocessing import RobustScaler,OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.compose import TransformedTargetRegressor

# Dataframe used is df_imputed, processing steps are embedded in a pipeline
target = 'pCR'
features_to_exlude_in_x = ['ID','pCR','RelapseFreeSurvival']


X,y = split_x_y(data_train,features_to_exlude_in_x,target)


X_test, y_test = split_x_y(data_test,features_to_exlude_in_x,target)
Xcols = X_test.columns.values

imputer = Pipeline(steps=[('knn_imputer',KNNImputer(n_neighbors=1))])
cv = RepeatedStratifiedKFold(n_repeats=3,n_splits=10,random_state=0)
classifier = XGBClassifier(scale_pos_weight = 0.78/0.22,n_jobs=-1) 
model = Pipeline(steps=[('imputer',imputer),('classifier',classifier)])


# Learning rate controls overfitting, the smaller learning rate means less correction is made to the preceding tree
# low value for learning rate  results into more trees
# `colsample_bytree` fraction of features to be used to build tree
# `gamma` minimum loss reduction to make split. It is pseudo-regularization hyperparameter for complexity control

param_grid = param_grid = {#'classifier__n_estimators': [100,200,300],
                            'classifier__max_depth': [3,4,5],
                            'classifier__learning_rate': [0.1,0.05,0.01],
                            'classifier__gamma': [0,0.25,1],
                            'classifier__colsample_bytree':[0.05,0.1,0.15]}

grid_search = GridSearchCV(estimator=model,param_grid=param_grid,cv=cv,scoring='balanced_accuracy')
grid_search.fit(X,y)
xgb = grid_search.best_estimator_
ba_xgb_cv = cross_val_score(xgb, X, y, cv=cv, scoring='balanced_accuracy')



y_pred = xgb.predict(X_test)
ba_xgb_test = balanced_accuracy_score(y_true=y_test,y_pred=y_pred)

print(f"Cross-validation accuracy: {ba_xgb_cv.mean():.2f}, SD: {ba_xgb_cv.std():.2f}")
print(f"Test accuracy: {ba_xgb_test:.2f}")
report=classification_report(y_true=y_test,y_pred=y_pred)
print(report)
xgb

In [ ]:
perfopmance_xgb = model_performance('XGBoost',ba_xgb_cv,ba_xgb_test)

In [ ]:
# plot_roc_curve(xgb)

In [ ]:
from sklearn.ensemble import VotingClassifier

clf_voting = VotingClassifier(estimators=[('random_forest',bal_random_forest),('logit',clf_lr), ('xgb',xgb)],voting='soft')
cv = RepeatedStratifiedKFold(random_state=0,n_repeats=3,n_splits=5)
ba_cv_voting = cross_val_score(clf_voting, X, y, cv=cv, scoring='balanced_accuracy')
clf_voting.fit(X=X,y=y)


y_pred = clf_voting.predict(X_test)
ba_test_voting = balanced_accuracy_score(y_true=y_test,y_pred=y_pred)
report=classification_report(y_true=y_test,y_pred=y_pred,output_dict=True)
print(f"CV BA {np.mean(ba_cv_voting):.3f} SD {np.std(ba_cv_voting):.3f} Test BA: {ba_test_voting:.3f}")
# print(report)
clf_voting

## SAVE BEST MODEL FOR TESTING

In [ ]:

import pickle

with open('clf_voting.pkl', 'wb') as f:
    pickle.dump(clf_voting, f)

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_true=y_test,y_pred=y_pred)
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
perfopmance_voting = model_performance('Voting Ensemble',ba_cv_voting,ba_test_voting)

In [ ]:
## comparison of model performance
model_name = []
cv_score = []
sd_cv_score = []
test_score = []
metrics_list = [perfopmance_clf_lr,perfopmance_bal_rf,perfopmance_xgb,perfopmance_voting]

for m in metrics_list:
    model_name.append(m['name'])
    cv_score.append(m['ba_cv'])
    sd_cv_score.append(m['sd'])
    test_score.append(m['ba_test'])


comparison = pd.DataFrame({'Model':model_name,'Cross Validation':cv_score, 'SD':sd_cv_score, 'Test':test_score})
print('Balanced Accuracy')
comparison

## EXPORT LATEX TABLE FOR REPORT

In [ ]:
import os
# Model comparison
try:
    # Create the directory if it doesn't exist
    dest_path = './Report_ML'
    os.makedirs(dest_path, exist_ok=True)

    # Convert DataFrame to LaTeX
    reg_mae = comparison.to_latex(float_format='%.3f',index=False)
    print(reg_mae)
    # Write the LaTeX string to the file
    with open('./Report_ML/clf_ba.tex', 'w') as f:
        f.write(reg_mae)
except Exception as e:
    print(e)


# Classification report
try:
    df = pd.DataFrame(report).transpose()
    indices = df.index.values
    indices2 = []
    for s in df.index.values:
        indices2.append(re.sub(r'\.0','',s))

    df.index = indices2
    
    # Export to LaTeX
    clf_report = df.to_latex(float_format='%.2f')

    print(clf_report)
    with open('./Report_ML/clf_report.tex', 'w') as f:
            f.write(clf_report)
except Exception as e:
    print(e)


In [ ]:
# df = pd.DataFrame(report).transpose()
indices = df.index.values
indices2 = []
for s in df.index.values:
    indices2.append(re.sub(r'\.0','',s))

indices2